In [ ]:
! pip install hazm

     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 1.4 MB 41.4 MB/s 
     |████████████████████████████████| 233 kB 45.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=dbb7a47c067effe7d96056505f206c94b4fddcf81c16bea3b24b317f64b2563d
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154638 sha256=ba0f1a508d8727927a2fb67ca0f89adaee8d8ed3a42ca0834b0ab710ade474d0
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from __future__ import unicode_literals
import csv
from hazm import *
import pandas as pd
from hazm.utils import *
import math
import itertools
from itertools import islice
import numpy as np
from numpy.linalg import norm
import time
import pickle
# from word_embedding1 import w2v_model
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

normalizer = Normalizer()

In [ ]:

normalizer = Normalizer()

exl = pd.ExcelFile('/content/drive/MyDrive/combined.xlsx')
# print(type(exl))


# with open('w2v.obj', 'rb') as f:
#     w2v_model = pickle.load(f)

# with open('doc_embedding.pkl', 'rb') as f:
#     doc_embedding = pickle.load(f)

df = exl.parse(exl.sheet_names[0])
# print(type(df))

stop_words = stopwords_list(default_stopwords)
punctuations = ['!', '؛', ':', '[', ']', '-', ')', '…', '»', 'انتهای', 'پیام/', '«', '(', '**', '*', '.', ',', '،', '؟',
                'گزارش', 'ایسنا']
stop_words += punctuations
# print(stop_words)
i = 0
doc_titles = {}
doc_topic = {}
contents = []
for index, row in df.iterrows():
    d1 = {}
    d1['doc_ID'] = i
    d1['tokens'] = word_tokenize(normalizer.normalize(row['content']))
    doc_titles[i] = row['url']
    doc_topic[i] = row['topic']
    contents.append(d1)
    i += 1


KeyboardInterrupt: ignored

In [ ]:

def get_doc_title(id):
    return doc_titles[id]


def get_doc_length(docID):
    c = contents[docID]
    terms = c['tokens']
    l = 0
    for t in terms:
        l += tfidf[t][1][docID] ** 2

    return l ** 0.5


def show_search_results(res, n):
    count = 0
    result = []
    for i in res.keys():
        if count < n:
            result.append(i)
            print(str(i) + " : " + get_doc_title(i))
            count += 1
    return result



In [ ]:
with open('/content/drive/MyDrive/contents.pkl', 'rb') as f:
    contents = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/doc_titles.pkl', 'rb') as f:
    doc_titles = pickle.load(f)

In [ ]:
N = len(contents)
N
stemmer = Stemmer()

In [ ]:

for c in contents:
    # print(type(c))
    temp = list(c['tokens'])
    for word in c['tokens']:
        # print(word)
        # iterating on a copy since removing will mess things up
        if word in stop_words:
            temp.remove(word)
    c['tokens'] = temp

stemmer = Stemmer()


for c in contents:
    temp = []
    for i in c['tokens']:
        temp.append(stemmer.stem(i))
        c['tokens'] = temp

N = len(contents)
# print(contents[10])




In [ ]:
def preprocess(phrase):
    # preprocess the user query
    phrase = normalizer.normalize(phrase).split()
    temp = list(phrase)
    for word in phrase:
        # print(word)
        # iterating on a copy since removing will mess things up
        if word in stop_words:
            temp.remove(word)
    phrase = temp
    # print(phrase)
    final_phrase = [stemmer.stem(p) for p in phrase]
    # print(final_phrase)

    return final_phrase



In [ ]:

pos_index = {}
inverted_index = {}
for c in contents:
    pos = 0
    for term in c['tokens']:
        if term in pos_index:
            if c['doc_ID'] not in inverted_index[term]:
                inverted_index[term].append(c['doc_ID'])
            pos_index[term][0] = pos_index[term][0] + 1
            if c['doc_ID'] in pos_index[term][1]:
                pos_index[term][1][c['doc_ID']].append(pos)

            else:
                pos_index[term][1][c['doc_ID']] = [pos]

        else:

            pos_index[term] = []
            pos_index[term].append(1)
            pos_index[term].append({})
            pos_index[term][1][c['doc_ID']] = [pos]
            inverted_index[term] = [c['doc_ID']]
        pos += 1

frequencies = {}
for i in list(pos_index.keys()):
    frequencies[i] = pos_index[i][0]


KeyboardInterrupt: ignored

In [ ]:
# now calculating the tf-idf, using postings list
K = 100
tfidf = {}
champ_list = {}
for i in inverted_index.keys():
    l = []
    dct = {}
    for j in inverted_index[i]:
        dct[j] = math.log(N / len(inverted_index[i]), 10) * (1 + math.log(len(pos_index[i][1][j]), 10)) # calculating the weight
    l.append(len(inverted_index[i]))
    l.append(dct)
    tfidf[i] = l
    champ_list[i] = dct

for k in champ_list.keys():
    tmp = dict(sorted(champ_list[k].items(), key=lambda item: item[1], reverse=True))
    champ_list[k] = dict(itertools.islice(tmp.items(), K))

NameError: ignored

In [ ]:
def get_weight(query):
    dic = {}
    qw = {}
    for term in query:
        if term in dic:
            dic[term] += 1
        else:
            dic[term] = 1
    
    for k in dic.keys():
        if k not in inverted_index:
            nt = 1
        else:
            nt = len(inverted_index[k])
        qw[k] = (1 + math.log(dic[k], 10)) * (math.log(N/nt, 10))

    return qw

In [ ]:
x = []
for c in contents:
    dc = {}
    id = c['doc_ID']
    for t in c['tokens']:
        tmp = tfidf[t][1][id] 
        dc[t] = tmp
    x.append(dc)

NameError: ignored

In [ ]:
import pickle
print(x[0])
with open('/content/drive/MyDrive/doc_tfidf.pkl', 'wb') as f:
    pickle.dump(x, f)

NameError: ignored

In [ ]:
with open('/content/drive/MyDrive/contents.pkl', 'wb') as f:
    pickle.dump(contents, f)

In [ ]:
with open('/content/drive/MyDrive/doc_titles.pkl', 'wb') as f:
    pickle.dump(doc_titles, f)

In [ ]:
with open('/content/drive/MyDrive/pos_index.pkl', 'wb') as f:
    pickle.dump(pos_index, f)

In [ ]:
with open('/content/drive/MyDrive/inverted_index.pkl', 'wb') as f:
    pickle.dump(inverted_index, f)

In [ ]:

model = Word2Vec.load("/content/drive/MyDrive/w2v_150k_hazm_300_v2.model")

with open('/content/drive/MyDrive/doc_tfidf.pkl', 'rb') as f:
    tfidf_docs = pickle.load(f)



In [ ]:
dembedding = []
# file_pi = open('w2v.obj', 'w')
# pickle.dump(w2v_model, file_pi)
for doc in tfidf_docs:
    v = np.zeros(300)
    w = 0
    # print(doc.items())
    for t, tw in doc.items():
        # print(t)
        if t in model.wv:
            v += model.wv[t] * tw
            w += tw
    dembedding.append(v / w)


In [ ]:
with open('d_embedding.pkl', 'wb') as f:
    pickle.dump(dembedding, f)

In [ ]:
def get_weight(query):
    dic = {}
    qw = {}
    for term in query:
        if term in dic:
            dic[term] += 1
        else:
            dic[term] = 1

    for k in dic.keys():
        if k not in inverted_index:
            nt = 1
        else:
            nt = len(inverted_index[k])
        qw[k] = (1 + math.log(dic[k], 10)) * (math.log(N / nt, 10))
    print(qw)

    return qw

Now we have prepared our data. we have our doc embedding and we can use this for k-means algorithm.

In [ ]:
with open('/content/drive/MyDrive/d_embedding.pkl', 'rb') as f:
    doc_embedding = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

In [ ]:
def get_query_vector(query):
    L = preprocess(query)
    qw = get_weight(L)
    v = np.zeros(300)
    w = 0
    for t, tw in qw.items():
        v += model.wv[t] * tw
        w += tw

    return v / w

Now let's dive in and do the k-means clustering!

In [ ]:
def get_similarity(doc1, doc2):
    score = np.dot(doc1, doc2) / (norm(doc1) * norm(doc2))
    return (score+1) * 2


In [ ]:
def init_centroids(docs, k):

    # centroids = [[0 for i in range(len(docs[0]))] for j in range(k)]
    centroids = [np.zeros(len(docs[0])) for j in range(k)]
    for n in range(k):
        random_index = np.random.randint(len(docs))
        centroids[n] = docs[random_index]

    return centroids 




In [ ]:
def update_assignment(centroids, docs, k):
    cluster = []
    cluster_sizes = [0 for i in range(k)]
    for i in range(len(docs)):
        max_similarity = 0
        cluster.append(max_similarity)
        for j in range(len(centroids)):
            sim = get_similarity(docs[i], centroids[j])
            if sim > max_similarity:
                cluster[i] = j
                max_similarity = sim
    for i in cluster:
        cluster_sizes[i] += 1

    return cluster, cluster_sizes



In [ ]:
centroids = init_centroids(doc_embedding, 7)
x, y = update_assignment(centroids, doc_embedding, 7)
print(x)
print(y)

[2, 5, 3, 5, 2, 5, 3, 3, 1, 1, 2, 2, 2, 3, 3, 3, 2, 5, 1, 2, 2, 1, 2, 1, 1, 1, 0, 3, 5, 5, 1, 5, 3, 5, 3, 5, 1, 0, 1, 3, 1, 2, 5, 0, 3, 5, 3, 1, 5, 2, 3, 3, 5, 5, 3, 3, 2, 2, 3, 0, 2, 2, 0, 2, 3, 3, 4, 3, 5, 3, 4, 3, 3, 1, 1, 0, 3, 3, 3, 3, 1, 1, 3, 2, 1, 3, 4, 2, 1, 2, 1, 3, 5, 3, 5, 1, 3, 2, 1, 0, 2, 2, 0, 1, 3, 3, 1, 3, 5, 3, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 2, 2, 0, 2, 3, 3, 2, 2, 0, 3, 1, 6, 0, 3, 0, 3, 5, 3, 0, 2, 0, 3, 1, 2, 2, 1, 5, 3, 3, 3, 5, 3, 3, 0, 3, 2, 2, 2, 1, 5, 3, 5, 2, 1, 1, 2, 0, 2, 3, 2, 1, 3, 3, 3, 2, 2, 5, 3, 0, 1, 3, 3, 4, 5, 3, 3, 0, 0, 0, 3, 3, 3, 0, 3, 3, 1, 0, 1, 2, 4, 3, 0, 0, 3, 3, 3, 2, 3, 5, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 5, 0, 3, 3, 0, 0, 3, 1, 1, 3, 5, 0, 0, 3, 3, 2, 5, 3, 2, 0, 0, 2, 5, 0, 3, 3, 1, 1, 3, 2, 2, 2, 3, 1, 3, 3, 3, 1, 1, 3, 5, 3, 0, 3, 0, 5, 1, 1, 0, 3, 2, 5, 1, 0, 2, 2, 4, 5, 3, 3, 0, 0, 2, 0, 3, 2, 3, 3, 0, 3, 1, 3, 6, 3, 0, 3, 0, 2, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 

In [ ]:
z = 0
for i in range(len(doc_embedding)):
    
    for j in doc_embedding[i]:
        if np.isnan(j):
            doc_embedding[i] = np.random.randn(len(doc_embedding[0]))


In [ ]:
def update_centroids(centroids, clusters, cluster_sizes, docs, k):
    new_centroids =  [np.zeros(len(docs[0])) for j in range(k)]
    # print(new_centroids[0])
    for j in range(len(docs)):
        # print(clusters[j])
        # if j < 10:
            # print(docs[j])
        new_centroids[clusters[j]] += docs[j]
    # print(new_centroids[0])
    print(cluster_sizes)
    for i in range(k):
        new_centroids[i] = new_centroids[i] / cluster_sizes[i]
        # print(new_centroids[i])

    return new_centroids


In [ ]:
def get_dist(a, b):
    x = 0
    for i in range(len(a)):
        x += (a[i] - b[i]) ** 2
    return x ** 0.5

In [ ]:
def RSS(centroids, clusters, docs):
    loss = 0
    for i in range(len(clusters)):
        loss = loss + get_similarity(centroids[clusters[i]], docs[i])

    return loss

In [ ]:
def k_means(docs, k):
    # randomly initializes k centroids
    centroids = init_centroids(docs, k)
    loss_list = []
    for i in range(20):
        clusters, cluster_sizes = update_assignment(centroids, docs, k)
        centroids = update_centroids(centroids, clusters, cluster_sizes, docs, k)
        loss = RSS(centroids, clusters, docs)
        if i:
            diff = np.abs(prev_loss - loss)
            if diff < 1e-16 and diff / prev_loss < 1e-16:
                break
        loss_list.append(loss)
        prev_loss = loss
        print('iter %d, loss: %.4f' % (i, loss))

    
    clusters_list = [[] for j in range(k)]
    for i in range(k):
        for c in range(len(clusters)):
            if clusters[c] == i:
                clusters_list[i].append(c)
        
            
        

    return centroids, clusters, clusters_list

        

In [ ]:
centroids, cluster_ids, clusters_list = k_means(doc_embedding, 7)

[4397, 2253, 653, 7704, 13223, 19662, 2169]
iter 0, loss: 157265.2243
[7781, 3093, 809, 6338, 13769, 16021, 2250]
iter 1, loss: 159214.5546
[8302, 3238, 927, 5995, 13918, 15072, 2609]
iter 2, loss: 159536.9394
[8463, 3357, 1119, 5842, 13915, 14554, 2811]
iter 3, loss: 159671.0987
[8344, 3467, 1433, 5751, 13844, 14261, 2961]
iter 4, loss: 159763.1584
[8061, 3609, 1872, 5726, 13678, 13988, 3127]
iter 5, loss: 159886.0122
[7548, 3769, 2465, 5833, 13451, 13730, 3265]
iter 6, loss: 160062.7800
[6954, 3993, 3092, 6066, 13172, 13413, 3371]
iter 7, loss: 160284.1747
[6497, 4206, 3623, 6333, 12850, 12968, 3584]
iter 8, loss: 160518.2665
[6330, 4375, 4008, 6545, 12561, 12465, 3777]
iter 9, loss: 160682.4460
[6443, 4503, 4262, 6646, 12426, 11911, 3870]
iter 10, loss: 160763.2230
[6684, 4552, 4438, 6675, 12357, 11401, 3954]
iter 11, loss: 160804.6634
[6997, 4587, 4589, 6678, 12316, 10884, 4010]
iter 12, loss: 160833.7769
[7297, 4588, 4742, 6682, 12298, 10393, 4061]
iter 13, loss: 160859.7783
[7545

In [ ]:
print(clusters_list[2])

[86, 90, 186, 192, 423, 611, 618, 797, 798, 811, 902, 1067, 1126, 1439, 1452, 1569, 1737, 1738, 1747, 1751, 1782, 1784, 1805, 1806, 1812, 1818, 1845, 1856, 1903, 1927, 1936, 1957, 1961, 1962, 2010, 2011, 2014, 2023, 2026, 2032, 2035, 2043, 2051, 2068, 2093, 2137, 2150, 2151, 2161, 2168, 2195, 2196, 2218, 2231, 2239, 2262, 2276, 2295, 2298, 2300, 2330, 2331, 2354, 2355, 2369, 2372, 2442, 2443, 2446, 2476, 2497, 2523, 2538, 2540, 2543, 2544, 2553, 2568, 2570, 2571, 2575, 2576, 2580, 2612, 2640, 2648, 2649, 2650, 2651, 2690, 2694, 2695, 2705, 2708, 2715, 2730, 2734, 2735, 2736, 2751, 2758, 2786, 2787, 2796, 2806, 2822, 2840, 2858, 2876, 2883, 2905, 2924, 2930, 2932, 2943, 2969, 2989, 3010, 3016, 3037, 3042, 3056, 3066, 3096, 3102, 3105, 3124, 3127, 3139, 3155, 3162, 3165, 3171, 3361, 3551, 3575, 3660, 3664, 3774, 4007, 5120, 5231, 5436, 5442, 5546, 5667, 5707, 5799, 5806, 5815, 5838, 5912, 6051, 6182, 6185, 6207, 6216, 6270, 6274, 6408, 6413, 6707, 6714, 6746, 6747, 6748, 6852, 6941, 6944

In [ ]:
def get_centers_similarity(query, centroids, b):
    similarities = []
    q = get_query_vector(query)
    for c in centroids:
        similarities.append(get_similarity(c, q))
    print(similarities)
    temp = sorted(similarities)[-b:]
    
    res = []
    for ele in temp:
      
        #  encapsulating elements with index using index()
        res.append(similarities.index(ele))
  
        # printing result 
    print("K Maximum with indices : " + str(res))

    return res, q



In [ ]:
print(get_centers_similarity("تیم فوتبال استقلال", centroids, 3))

In [ ]:
def search(query, docs, clusters, centroids, b):
    scores = {}
    if query.__contains__('cat:'):
        t = query.split(':')
        topic = t[1].split()[0]
        query = t[1].split()[1]
        q = get_query_vector(query)
        print(topic)
        for k in topics[topic]:
            scores[k] = get_similarity(q, docs[k])
    else:
        b_closest, q = get_centers_similarity(query, centroids, b)
        for bc in b_closest:
            for c in clusters[bc]:
                scores[c] = get_similarity(q, docs[c])

    
    return dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

In [ ]:
import time
while True:
    query = input("enter query: ")
    start = time.time()
    res = search(query, doc_embedding, clusters_list, centroids, 3)
    print(time.time() - start)
    show_search_results(res, 10)


enter query: دانشگاه
[2.8017212889698344, 2.234517762858726, 2.646343454122773, 1.9142207680522016, 2.2139424700853003, 2.561702800691488, 2.6098585198842326]
K Maximum with indices : [6, 2, 0]
0.418872594833374
45671 : https://www.farsnews.ir/news/13990917000273/وزیر-علوم-برای-عدم-نظارت-بر-دانشگاه‌های-علمی-کاربردی-تذکر-گرفت
49563 : https://www.farsnews.ir/news/13990623000349/آغاز-رسمی-سال-تحصیلی-دانشگاه‌ها-با-حضور-مجازی-روحانی
7176 : https://www.isna.ir/news/98120302440/مدارس-و-دانشگاه-های-البرز-برای-پیشگیری-از-کرونا-تعطیل-شد-خبر
48273 : https://www.farsnews.ir/news/13990721000619/علی-مظفری-مسئول-مرکز-هماهنگی-بسیج-دانشجویی-دانشگاه-آزاد-شد
10252 : https://www.isna.ir/news/98053014683/رشته-نوازندگی-پیانو-ایرانی-به-مقطع-کارشناسی-اضافه-شد
41255 : https://www.farsnews.ir/news/13991218000610/استفاده-غیرقانونی-فرزند-معاون-وزیر-علوم-از-سهمیه-استعدادهای-درخشان
9224 : https://www.isna.ir/news/99110906710/پیامی-برای-درگذشت-سیدمحمود-شریعت
41962 : https://www.farsnews.ir/news/13991205000460/معاون-

KeyboardInterrupt: ignored

In [ ]:
def get_knn(docs, k, query):
    # q = get_query_vector(query)
    scores = {}
    for doc in range(len(docs)):
        scores[doc] = get_similarity(docs[doc], query)

    temp =  dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))
    res = {}
    count = 0
    for i in temp.keys():
        if count < k:
            res[i] = doc_topic[i]
            count += 1

    return res
        

In [ ]:
with open('doc_embedding_7.pkl', 'rb') as f:
    doc_embedding_7k = pickle.load(f)

In [ ]:
from tqdm import tqdm
from collections import Counter 
def set_cat(docs1, docs2, k):
    doc_topics = {}
    for s in tqdm(range(len(docs2))):
        top_k = get_knn(docs1, k, docs2[s])
        # print(top_k)
        value, count = Counter(top_k.values()).most_common(1)[0]
        # print(value)
        doc_topics[s] = value

    return doc_topics

            


In [ ]:
doc_topics_7k = set_cat(doc_embedding, doc_embedding_7k, 5)

  0%|          | 0/7562 [00:01<?, ?it/s]


KeyError: ignored

In [ ]:
print(doc_topics_7k)
with open('knn_7k.pkl', 'wb') as f:
    pickle.dump(doc_topics_7k, f)

{0: 'sport', 1: 'sport', 2: 'sport', 3: 'sport', 4: 'sport', 5: 'sports', 6: 'sport', 7: 'sport', 8: 'sport', 9: 'sport', 10: 'sport', 11: 'sport', 12: 'sport', 13: 'sport', 14: 'sport', 15: 'sport', 16: 'sport', 17: 'sport', 18: 'sport', 19: 'sport', 20: 'sport', 21: 'sport', 22: 'sport', 23: 'sport', 24: 'sport', 25: 'sport', 26: 'sport', 27: 'sport', 28: 'sport', 29: 'sport', 30: 'sport', 31: 'sport', 32: 'sport', 33: 'sport', 34: 'sport', 35: 'sport', 36: 'sport', 37: 'sport', 38: 'sport', 39: 'sport', 40: 'sport', 41: 'sport', 42: 'sport', 43: 'sport', 44: 'sport', 45: 'sport', 46: 'sport', 47: 'sport', 48: 'sport', 49: 'sport', 50: 'sport', 51: 'sport', 52: 'sport', 53: 'sport', 54: 'sport', 55: 'sport', 56: 'sport', 57: 'sport', 58: 'sport', 59: 'sport', 60: 'sport', 61: 'sport', 62: 'sport', 63: 'sports', 64: 'sports', 65: 'sport', 66: 'sport', 67: 'sport', 68: 'sport', 69: 'sport', 70: 'sports', 71: 'sport', 72: 'sport', 73: 'sport', 74: 'sport', 75: 'sport', 76: 'sport', 77: 

In [ ]:
with open('knn_7k.pkl', 'rb') as f:
    doc_topics_7k = pickle.load(f)

In [ ]:
topics = {'sport':[], 'economy':[], 'culture':[], 'health':[], 'political':[]}
for i in doc_topics_7k.keys():
    if doc_topics_7k[i] == 'sports' or doc_topics_7k[i] == 'sport':
        topics['sport'].append(i)
    elif doc_topics_7k[i] == 'political' or doc_topics_7k[i] == 'politics':
        topics['political'].append(i)
    else:
        topics[doc_topics_7k[i]].append(i)

